In [33]:
import numpy as np
import pandas as pd
import glob
from time import sleep
from tqdm import tqdm
import time
import signal
import selenium
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from time import sleep
import os
import pickle
import json

In [22]:
site="https://powszechswiat.pl/user"
query="kościół +wirus"

In [29]:
def save_cookie(d, path):
    with open(path, 'wb') as filehandler:
        pickle.dump(d.get_cookies(), filehandler)

In [34]:
with open("cookies.json", 'w') as filehandler:
    json.dump(d.get_cookies(), filehandler)

In [31]:
d = webdriver.Chrome()

d.get(site)



sleep(60)

save_cookie(d, 'cookie.pickle')
# d.find_element(By.XPATH, '//div[@id="edit-actions"]').click()

d.get(f"https://www.tygodnikpowszechny.pl/search/site/{query}")

pagesMax=d.find_element(By.XPATH, '//li[@class="pager-last last"]/a').get_attribute('href')
pagesMax=int(pagesMax.split('=')[-1])
print(f'Found {pagesMax} pages')

links=[]

els=d.find_elements(By.XPATH, '//li[@class="search-result"]//a')
for el in els:
    links.append(el.get_attribute('href'))

for page in tqdm(range(pagesMax)):
    site=f"https://www.tygodnikpowszechny.pl/search/site/{query}?page={page+1}"
    d.get(site)
    els=d.find_elements(By.XPATH, '//li[@class="search-result"]//a')
    for el in els:
        links.append(el.get_attribute('href'))

Found 3 pages


100%|██████████| 3/3 [00:06<00:00,  2.09s/it]


In [36]:
def load_cookie(d, path):
    with open(path, 'r') as cookiesfile:
        cookies = json.load(cookiesfile)
    for cookie in cookies:
        d.add_cookie(cookie)

In [41]:
d = webdriver.Chrome()

titles=[]
leads=[]
txts=[]
auths=[]
added=[]

for link in tqdm(links[:5], desc='Getting articles'):    
    try:
        d.get(link)
        load_cookie(d,"cookies.json")
    except WebDriverException:
        print('WebDriverException, link: ', link)
        continue
    try:
        titles.append(d.find_element(By.XPATH, '//h1[@class="field-content"]').text)
    except:
        titles.append(np.nan)
    try:
        leads.append(d.find_element(By.XPATH, '//div[@class="views-field views-field-field-summary"]/div[@class="field-content"]').text)
    except:
        leads.append(np.nan)
    try:
        txts.append(d.find_element(By.XPATH, '//div[@class="views-field views-field-body"]/div[@class="field-content"]').text)
    except:
        txts.append(np.nan)
    try:
        auths.append(d.find_element(By.XPATH, '//div[@class="field-content views-field-name"]').text)
    except:
        auths.append(np.nan)
    try:
        added.append(d.find_element(By.XPATH, '//span[@class="field-content"]').text)
    except:
        added.append(np.nan)

    sleep(2)

# articles to df
# df=pd.DataFrame({'title':titles, 'leads':leads, 'txt':txts, 'auth':auths, 'added':added, 'links':links[:2]})
# df.to_csv(f'data/arts/tp_{query}.csv', index=False)
# df.to_excel(f'data/arts/tp_{query}.xlsx', index=False)


Getting articles: 100%|██████████| 5/5 [00:16<00:00,  3.29s/it]

WebDriverException, link:  https://www.tygodnikpowszechny.pl/kosciol-w-skrocie-163720
WebDriverException, link:  https://www.tygodnikpowszechny.pl/bylam-przy-poczatku-i-koncu-163080


In [16]:
df

,title,leads,txt,auth,added,links
0,WIRUS I WIELKA TAJEMNICA WIARY,Epidemia przerwała tradycyjny sposób celebracj...,Dzieci ubrane na biało zmierzają do parafialny...,PIOTR SIKORA,04.05.2020,https://www.tygodnikpowszechny.pl/wirus-i-wiel...
1,WIRUS UDERZA W FINANSE KOŚCIOŁA,Powodowana pandemią nieobecność ludzi na niedz...,Skąd czerpać teraz środki na utrzymanie budynk...,ARTUR SPORNIAK,23.03.2020,https://www.tygodnikpowszechny.pl/wirus-uderza...


In [10]:
df['txt'][0]

'Dzieci ubrane na biało zmierzają do parafialnych kościołów. Chłopcy w garniturach albo – coraz częściej – dziwnych kostiumach: jakby liturgiczna alba, długa jak koszula, do tego spodnie; dziewczynki – w sukniach księżniczek, jakby do ślubu. Wielkie styropianowe chleby i winne grona na skraju kościelnych prezbiteriów. W domach weselnych i restauracjach przyjęcia, których głównymi bohaterami są dziesięciolatkowie. Przy drogach reklamy zestawów komputerowych z nieco eucharystyczną estetyką.\nRozczarowanie\nWrosły już te obrazy w polski pejzaż majowy, ale w tym roku, z powodu epidemii COVID-19, ich nie ma. Już od marca, gdy tylko szkoły zostały zamknięte, a liczba uczestników celebracji liturgicznych radykalnie ograniczona, kurie diecezjalne zasypywane były pytaniami, co w tym roku będzie z pierwszokomunijnymi uroczystościami. Wobec niejasnego stanowiska rządu co do tego,...'